In [1]:
# CODE FOR CREATING AND TRAINING CHATBOT

# import libraries
import random
import json
import pickle
import numpy as np
import nltk
nltk.download('punkt')
nltk.download('wordnet')

from keras.models import Sequential
from nltk.stem import WordNetLemmatizer
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD

[nltk_data] Downloading package punkt to /home/ata/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/ata/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
2023-11-27 13:06:48.207506: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-27 13:06:48.232807: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-27 13:06:48.232840: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-27 13:06:48.233913: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already bee

In [2]:
# load json file with tags, patterns & responses

intents = json.loads(open("intents.json").read())

In [3]:
# create lists to store data
words = []
labels = []
docs = []
ignore_letters = ["?", "!", ".", ","]

# separate words from patterns and add to words list
for intent in intents['intents']:
	for pattern in intent['patterns']:
		word_list = nltk.word_tokenize(pattern)
		words.extend(word_list) 
		
		# associate patterns with respective tags
		docs.append(((word_list), intent['tag']))

		# append tags to labels list
		if intent['tag'] not in labels:
			labels.append(intent['tag'])

In [4]:
# store root words (lemma)
lemmatizer = WordNetLemmatizer()

words = [lemmatizer.lemmatize(word)
		for word in words if word not in ignore_letters]
words = sorted(set(words))

In [5]:
# save words and labels lists to binary files

pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(labels, open('labels.pkl', 'wb'))

In [6]:
# binarise data for neural network processing:
# (0 = word not in pattern, 1 = word in pattern)

training = []
output_empty = [0]*len(labels)
for doc in docs:
	bag = []
	word_patterns = doc[0]
	word_patterns = [lemmatizer.lemmatize(
		word.lower()) for word in word_patterns]
	for word in words:
		bag.append(1) if word in word_patterns else bag.append(0)
  # shuffle training data
	output_row = list(output_empty)
	output_row[labels.index(doc[1])] = 1
	training.append([bag, output_row])
random.shuffle(training)
# Change starts here
random.shuffle(training)

train_x = []
train_y = []

for pair in training:
    train_x.append(pair[0])
    train_y.append(pair[1])

In [7]:
# split data using np array
train_x = np.array(train_x)
train_y = np.array(train_y)

In [8]:
# create Sequential Neural Network model
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]), ),
				activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

2023-11-27 13:06:50.367929: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-27 13:06:50.368880: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [9]:
# compile model
sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
			optimizer=sgd, metrics=['accuracy'])
hist = model.fit(np.array(train_x), np.array(train_y),
				epochs=200, batch_size=5, verbose=1)

print("Training complete")

Epoch 1/200
23/23 [==============================] - 0s 857us/step - loss: 3.7447 - accuracy: 0.0442
Epoch 2/200
23/23 [==============================] - 0s 691us/step - loss: 3.6112 - accuracy: 0.0354
Epoch 3/200
23/23 [==============================] - 0s 735us/step - loss: 3.5287 - accuracy: 0.1150
Epoch 4/200
23/23 [==============================] - 0s 712us/step - loss: 3.4907 - accuracy: 0.0796
Epoch 5/200
23/23 [==============================] - 0s 739us/step - loss: 3.3468 - accuracy: 0.1947
Epoch 6/200
23/23 [==============================] - 0s 669us/step - loss: 3.2041 - accuracy: 0.1504
Epoch 7/200
23/23 [==============================] - 0s 647us/step - loss: 3.1933 - accuracy: 0.1858
Epoch 8/200
23/23 [==============================] - 0s 663us/step - loss: 2.9862 - accuracy: 0.2478
Epoch 9/200
23/23 [==============================] - 0s 650us/step - loss: 2.8573 - accuracy: 0.2655
Epoch 10/200
23/23 [==============================] - 0s 700us/step - loss: 2.7899 - accura

In [10]:
# saving the model
model.save("chatbotmodel.h5", hist)

/home/ata/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
